In [147]:
import numpy as np
import decimal
from matplotlib import pyplot
import matplotlib.pyplot as plt
import math
from scipy.stats import norm
from scipy.integrate import quad
np.set_printoptions(precision=2)
import scipy
import pandas as pd

# BSM

In [109]:
import math
from scipy.stats import norm
import numpy as np


def blackScholesMertonCallPrice(time: float, underlying: float, strike: float, interest: float, sigma: float) -> float:
    if time == 0:
        time = 0.001
    if underlying == 0:
        underlying = 0.001
    if sigma == 0:
        sigma = 0.01
    d1 = (math.log(underlying / strike, math.e) + (interest + (sigma * sigma) / 2) * time) / (sigma * math.sqrt(time))
    d2 = d1 - sigma * math.sqrt(time)
    nd1 = norm.cdf(d1)
    nd2 = norm.cdf(d2)
    callPrice = underlying * nd1 - strike * math.exp(-interest * time) * nd2
    return callPrice


def matrixBlackScholes(t_grid: list, s_grid: list, strike: float, interest: float, sigma: float) -> list[list]:
    # grid = np.zeros(shape=(len(s_grid), len(t_grid)))

    grid = [[0 for j in range(len(t_grid))] for i in range(len(s_grid))]
    for i in range(len(s_grid)):
        for j in range(len(t_grid)):
            grid[i][j] = round(blackScholesMertonCallPrice(t_grid[i], s_grid[j], strike, interest, sigma), 10)
    return grid


# SABR

In [111]:
def zeta(vega, alpha, f, K, beta):
    term1 = vega / alpha
    term2 = pow(f * K, (1 - beta) * 0.5)
    term3 = math.log(f / K)
    return term1 * term2 * term3


def x_zeta_function(z, rho):
    denominator = 1 - rho
    numerator = math.sqrt(1 - 2 * rho * z + z ** 2) + z - rho
    res = math.log(numerator / denominator)
    return res


def term1(f, K, beta, alpha):
    denominator_term1 = pow(f * K, (1 - beta) * 0.5) * (
            1 + (pow(1 - beta, 2) / 24) * pow(math.log(f / K), 2) + (pow(1 - beta, 4) / 1920) * pow(math.log(f / K),
                                                                                                    4))
    term1 = alpha / denominator_term1
    return term1


def term2(z, rho):
    denominator = x_zeta_function(z, rho)
    term2 = z / denominator
    return term2


def term3(alpha, beta, rho, vega, f, K, timeExp):
    t1 = (pow(1 - beta, 2) / 24) * (alpha ** 2 / pow(f * K, 1 - beta))
    t2 = 0.25 * (rho * beta * vega * alpha) / (pow(f * K, (1 - beta) * 0.5))
    t3 = 1 / 24 * (2 - 3 * rho) ** 2 * vega ** 2
    res = 1 + timeExp * (1 + t1 + t2 + t3)
    return res

def sabrImpliedVolatility(alpha, beta, rho, vega, strike, time, underlying_value):
#     strike = int(strike)
    if(underlying_value == 0):
        return 0
    t1 = term1(underlying_value, strike, beta, alpha)
    t2 = term2(zeta(vega, alpha, underlying_value, strike, beta), rho)
    t3 = term3(alpha, beta, rho, vega, underlying_value, strike, time)
    implied_volatility = t1 * t2 * t3
    return implied_volatility


def BSM_matrix_SABR(t_grid, s_grid, strike, r, alpha, beta, rho, vega):
    matrix = list()
    for i in range(len(t_grid)):
        row = np.array([0 for i in range(len(s_grid))])
        for j in range(len(s_grid)):
            sigma = sabrImpliedVolatility(alpha,beta, rho, vega, strike, t_grid[i], s_grid[j])
            ans = calcBSM(t_grid[i], s_grid[j], strike, r, sigma)
            if (math.isnan(ans)):
                ans = 0
            row[j] = ans
        matrix.append(row)
    return matrix


# Heston

In [110]:
import numpy as np
i = complex(0, 1)


def product(rho: float, sigma: float, phi: complex) -> complex:
    return rho * sigma * i * phi


def d_f(pr: complex, kappa: float, phi: complex, sigma: float) -> complex:
    term1 = (pr - kappa) ** 2
    term2 = (sigma ** 2) * (i * phi + phi ** 2)
    return np.sqrt(term1 + term2)


def g_f(kappa: float, pr: complex, d: complex) -> complex:
    numerator = kappa - pr - d
    denominator = kappa - pr + d
    return numerator / denominator


def charFterm1(phi: complex, g: complex, d: complex, sigma: float, kappa: float, theta: float, interest: float,
               time: float,
               underlying: float) -> float:
    p1 = underlying ** (i * phi) * np.exp(i * phi * interest * time)
    numerator = 1 - g * np.exp(-d * time)
    denominator = 1 - g
    return p1 * (numerator / denominator) ** (-2 * theta * kappa / (sigma ** 2))


def charFterm2(theta: float, kappa: float, time: float, sigma: float, pr: complex, d: complex, sigmasigma: float,
               g: complex) -> float:
    p1 = theta * kappa * time / (sigma ** 2)
    p2 = kappa - pr - d
    p3 = sigmasigma / (sigma ** 2)
    p4 = (1 - np.exp(-d * time)) / (1 - g * np.exp(-d * time))
    return np.exp((p1 * p2) + (p3 * p2 * p4))


def hestonCharacteristicFunction(phi: complex, underlying: float, strike: float, interest: float, time: float,
                                 sigma: float, kappa: float, theta: float, sigmasigma: float, rho: float) -> complex:
    pr = product(rho, sigma, phi)
    d = d_f(pr, kappa, phi, sigma)
    g = g_f(kappa, pr, d)
    term1 = charFterm1(phi, g, d, sigma, kappa, theta, interest, time, underlying)
    term2 = charFterm2(theta, kappa, time, sigma, pr, d, sigmasigma, g)
    return (term1 * term2)


def hestonPrice(underlying: float, strike: float, interest: float, time: float, sigma: float, kappa: float,
                theta: float, sigmasigma: float, rho: float) -> float:
    price, iter, maxN = 0, 1000, 100
    dphi = maxN / iter
    if underlying == 0:
        underlying = 0.001
    if time == 0:
        time = 0.001

    # Calculate the complex integral
    # Using j instead of i to avoid confusion
    for j in range(1, iter):
        phi1 = dphi * (2 * j + 1) / 2
        phi2 = phi1 - i

        numerator1 = hestonCharacteristicFunction(phi2, underlying, strike, interest, time, sigma, kappa, theta,
                                                  sigmasigma, rho)
        numerator2 = strike * hestonCharacteristicFunction(phi1, underlying, strike, interest, time, sigma, kappa,
                                                           theta, sigmasigma, rho)
        denominator = np.exp(np.log(strike) * i * phi1) * i * phi1

        price = price + dphi * (numerator1 - numerator2) / denominator

    term1 = 0.5 * (underlying - strike * np.exp(-interest * time))
    term2 = price / np.pi

    return np.real((term1 + term2))


def matrixHeston(t_grid: list, s_grid: list, strike: float, interest: float, sigma: float, kappa: float, theta: float,
                 sigmasigma: float, rho: float) -> list[list]:
    # grid = np.zeros(shape=(len(s_grid), len(t_grid)))

    grid = [[0 for j in range(len(t_grid))] for i in range(len(s_grid))]
    for i in range(len(s_grid)):
        for j in range(len(t_grid)):
            grid[i][j] = round(hestonPrice(s_grid[i], strike, interest, t_grid[j], sigma, kappa, theta, sigmasigma, rho), 2)
            if(grid[i][j] < 0):
                grid[i][j] = 0
    return grid


# Calibrating

In [177]:
#
# Data preprocessing
#
df = pd.read_csv('aapl_2021_2023.csv')
df.columns = df.columns.str.strip()
df.pop("C_DELTA")
df.pop("C_GAMMA")
df.pop("C_VEGA")
df.pop("C_THETA")
df.pop("C_RHO")
df.pop("C_VOLUME")
df = df[df != " "].dropna(axis=0)
n = 100
dffilter = df.EXPIRE_DATE.str.contains('^2023')
df = df[dffilter]
df = df.sample(n=1000)
df.index = range(len(df))
S0 = pd.to_numeric(df["UNDERLYING_LAST"])
K = pd.to_numeric(df["STRIKE"])
tau = pd.to_numeric(df["DTE"])
P = pd.to_numeric(df["C_BID"])
volatilities_data = pd.to_numeric(df["C_IV"])
df["interest"] = 0.15
r = pd.to_numeric(df["interest"])

/var/folders/xz/p8m06xr55dv4n40tlgm3jl880000gn/T/ipykernel_1714/1141778691.py:4: DtypeWarning: Columns (14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('aapl_2021_2023.csv')


In [178]:
#
# Calibrating Heston Model
#
params = {
          "sigma": {"x0": 0.1, "lbub": [1e-3, 10]},
          "kappa": {"x0": 1., "lbub": [1e-3, 10]},
          "theta": {"x0": 0.1, "lbub": [1e-3, 10]},
          "sigmasigma": {"x0": 0.5, "lbub": [1e-2, 10]},
          "rho": {"x0": -0.5, "lbub": [-1, 0]},
          }
x0 = [param["x0"] for key, param in params.items()]
bnds = [param["lbub"] for key, param in params.items()]


#Objective Function
def squared_error(x):
    sigma, kappa, theta, sigmasigma, rho = [parameter for parameter in x]
    error = 0
    for i in range(len(S0)):
        left = float(P[i])
        right = hestonPrice(float(S0[i]), float(K[i]), float(r[i]), float(tau[i]), float(sigma), float(kappa), float(theta), float(sigmasigma), float(rho))
        error += (left - right)**2
    error / len(S0)
    print(error)
    pen = 0
    return error + pen


# Using scipy optimize
result = scipy.optimize.minimize(squared_error, x0, tol=1e-3, method='SLSQP', options={'maxiter': 1e6}, bounds=bnds)
v0, kappa, theta, sigma, rho = [param for param in result.x]
print(v0, kappa, theta, sigma, rho)

6.822771781036783e+107
6.822772117975514e+107
6.822771750274846e+107
6.822765950585031e+107
6.822771773837454e+107
6.822771712444147e+107
0.1 1.0 0.1 0.5 -0.5


In [180]:
#
#Calibrating SABR model
#
paramsSABR = {"alpha": {"x0": 0.01, "lbub": [1e-3,2]}, 
          "beta": {"x0": 0.05, "lbub": [1e-3,1]},
          "rho": {"x0": 0.01, "lbub": [-1,1]},
          "vega": {"x0": 0.001, "lbub": [0,10]},
          }
y0 = [paramSABR["x0"] for key, paramSABR in paramsSABR.items()]
bnds = [paramSABR["lbub"] for key, paramSABR in paramsSABR.items()]

# Objective function
def SqErr(x):
    alpha, beta, rho, vega = [param for param in x]
    
    err = 0
    for i in range(len(volatilities_data)):
        err += (volatilities_data[i] - sabrImpliedVolatility(alpha, beta, rho, vega, K[i], tau[i], S0[i]))**2
        err = P[i] - blackScholesMertonCallPrice(tau[i], S0[i], K[i], r[i],volatilities_data[i] )
    err / len(volatilities_data)
    pen = 0
    print(err)
    return err + pen

#Using scipy optimize
result = scipy.optimize.minimize(SqErr,y0, tol = 1, method='SLSQP', options={'maxiter': 1e4 }, bounds=bnds)
alpha, beta, rho, vega = [param for param in result.x]
print(alpha,beta, rho, vega)
                

-153.95
-153.95
-153.95
-153.95
-153.95
0.01 0.05 0.01 0.001


/var/folders/xz/p8m06xr55dv4n40tlgm3jl880000gn/T/ipykernel_1714/3311698924.py:25: RuntimeWarning: invalid value encountered in double_scalars
  term2 = z / denominator


# Comparing

In [231]:
# Heston calibrated parameters
sigmaHeston = 0.4
kappa = 5  # rate of mean reversion of variance process
theta = 0.001  # long-term mean variance
sigmasigma = 0.018 # volatility of volatility
lambd = 0.6  # risk premium of variance
rhoHeston = -0.999  # correlation between variance and stock process
    
# SABR calibrated parameters
alpha = 0.01
beta = 0.4501
rhoSABR = 0.1
vega = 0.1

# Take another sample from the data
# Pre-process data frame
df = pd.read_csv('aapl_2021_2023.csv')
df.columns = df.columns.str.strip() 
df.pop("C_DELTA")
df.pop("C_GAMMA")
df.pop("C_VEGA")
df.pop("C_THETA")
df.pop("C_RHO")
df.pop("C_VOLUME")
df = df[df != " "].dropna(axis=0)
df = df.loc[~(df==0).all(axis=1)]
n = 50
dffilter = df.EXPIRE_DATE.str.contains('^2021') 
df = df[dffilter]
df = df.sample(n=50)
df.index = range(len(df))
df.head()

/var/folders/xz/p8m06xr55dv4n40tlgm3jl880000gn/T/ipykernel_1714/1749978242.py:17: DtypeWarning: Columns (14,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('aapl_2021_2023.csv')


,QUOTE_UNIXTIME,QUOTE_READTIME,QUOTE_DATE,UNDERLYING_LAST,EXPIRE_DATE,EXPIRE_UNIX,DTE,C_IV,C_LAST,C_SIZE,C_BID,C_ASK,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT
0,1617998400,2021-04-09 16:00,2021-04-09,133.00,2021-05-14,1621022400,35.00,0.294950,3.52,396 x 11,3.51,3.61,136.00,3.0,0.023
1,1622577600,2021-06-01 16:00,2021-06-01,124.31,2021-08-20,1629489600,80.00,1.154540,67.25,122 x 164,64.3,64.5,60.00,64.3,0.517
2,1625860800,2021-07-09 16:00,2021-07-09,145.10,2021-11-19,1637355600,133.04,0.280260,12.3,3 x 34,12.3,12.4,140.00,5.1,0.035
3,1611781200,2021-01-27 16:00,2021-01-27,142.53,2021-06-18,1624046400,141.96,1.242440,111.43,237 x 100,120.8,121.95,21.25,121.3,0.851
4,1615410000,2021-03-10 16:00,2021-03-10,119.97,2021-04-01,1617307200,21.96,0.423060,0.15,6 x 272,0.14,0.16,147.00,27.0,0.225


In [232]:
#
#  Helper functions
#
def BSMrow(row):
    return (row[11] - blackScholesMertonCallPrice(row[6]/365, row[3], row[12], 0.1, 0.4))**2

def HestonRow(row):
    return (row[11] - hestonPrice(row[3], row[12],0.1,row[6]/365, sigmaHeston, kappa, theta, sigmasigma, rhoHeston))**2

def SABRrow(row):
    implied_V = sabrImpliedVolatility(alpha, beta, rhoSABR, vega, row[12], row[6]/365, row[3])
    return (row[11] - calcBSM(row[6]/365, row[3], row[12], 0.1, implied_V) ) **2

# Calculate price estimated by each model
df['BSM_price'] = df.apply(BSMrow, axis=1)
df['Heston_price'] = df.apply(HestonRow, axis=1)
df['SABR_price'] = df.apply(SABRrow, axis=1)
print(df['BSM_price'].sum())
print(df['Heston_price'].sum())
print(df['SABR_price'].sum())


142.61029628196968
100.82976204126483
85.98289438766078


In [234]:
df.head()

,QUOTE_UNIXTIME,QUOTE_READTIME,QUOTE_DATE,UNDERLYING_LAST,EXPIRE_DATE,EXPIRE_UNIX,DTE,C_IV,C_LAST,C_SIZE,C_BID,C_ASK,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,BSM_price,Heston_price,SABR_price
0,1617998400,2021-04-09 16:00,2021-04-09,133.00,2021-05-14,1621022400,35.00,0.294950,3.52,396 x 11,3.51,3.61,136.00,3.0,0.023,4.768570,6.689731,13.032100
1,1622577600,2021-06-01 16:00,2021-06-01,124.31,2021-08-20,1629489600,80.00,1.154540,67.25,122 x 164,64.3,64.5,60.00,64.3,0.517,1.234036,0.096709,1.233791
2,1625860800,2021-07-09 16:00,2021-07-09,145.10,2021-11-19,1637355600,133.04,0.280260,12.3,3 x 34,12.3,12.4,140.00,5.1,0.035,45.014754,1.293510,5.239406
3,1611781200,2021-01-27 16:00,2021-01-27,142.53,2021-06-18,1624046400,141.96,1.242440,111.43,237 x 100,120.8,121.95,21.25,121.3,0.851,0.019772,27.327955,0.019772
4,1615410000,2021-03-10 16:00,2021-03-10,119.97,2021-04-01,1617307200,21.96,0.423060,0.15,6 x 272,0.14,0.16,147.00,27.0,0.225,0.002769,0.064256,0.025600
